In [3]:
import polars as pl
import torch
import torch.nn as nn
import xgboost as xgb
import lightgbm as lgb


In [4]:
df=pl.read_csv('train.csv')

df.head()

df.describe()



statistic,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
str,f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
"""count""",891.0,891.0,891.0,"""891""","""891""",714.0,891.0,891.0,"""891""",891.0,"""204""","""889"""
"""null_count""",0.0,0.0,0.0,"""0""","""0""",177.0,0.0,0.0,"""0""",0.0,"""687""","""2"""
"""mean""",446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null
"""std""",257.353842,0.486592,0.836071,null,null,14.526497,1.102743,0.806057,null,49.693429,null,null
"""min""",1.0,0.0,1.0,"""Abbing, Mr. Anthony""","""female""",0.42,0.0,0.0,"""110152""",0.0,"""A10""","""C"""
"""25%""",224.0,0.0,2.0,null,null,20.0,0.0,0.0,null,7.925,null,null
"""50%""",446.0,0.0,3.0,null,null,28.0,0.0,0.0,null,14.4542,null,null
"""75%""",669.0,1.0,3.0,null,null,38.0,1.0,0.0,null,31.0,null,null
"""max""",891.0,1.0,3.0,"""van Melkebeke, Mr. Philemon""","""male""",80.0,8.0,6.0,"""WE/P 5735""",512.3292,"""T""","""S"""


In [5]:
X=df.drop(['Survived','PassengerId'])
y=df['Survived']

## Convert every non numeric column to categorical
for col in X.columns:
    if X[col].dtype == pl.String:
        print(X[col].dtype)
        X = X.with_columns(pl.col(col).cast(pl.Categorical))




String
String
String
String
String


In [6]:
X=X.drop("Name")




In [7]:
X_xgb=X.to_pandas()
y_xgb=y.to_pandas()


In [8]:
for col in X_xgb.columns:
    print(X_xgb[col].dtype)


int64
category
float64
int64
int64
category
float64
category
category


In [9]:
X_xgb.drop(columns=['Cabin',"Fare","Ticket","Age","Embarked"],inplace=True,errors='ignore')

In [10]:
X_xgb['Sex']=X_xgb["Sex"].map({"male":0,"female":1})

In [11]:
pos_weight=len(y_xgb[y_xgb==0])/len(y_xgb[y_xgb==1])
pos_weight

1.605263157894737

In [12]:
import optuna
from sklearn.model_selection import cross_val_score


def objective(trial: optuna.Trial):
    params = {
        'objective': 'binary:logistic',
        'eta': trial.suggest_float('eta', 0.001, 0.3,log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'gamma': trial.suggest_float('gamma', 0.1, 1,log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 100,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 100,log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10,log=True),
        'enable_categorical': True
    }   

    model = xgb.XGBClassifier(**params)
    cv_scores = cross_val_score(model, X_xgb, y_xgb, cv=5, scoring='accuracy')
    return cv_scores.mean()

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(objective, n_trials=100)

[I 2025-04-01 01:30:52,525] A new study created in memory with name: no-name-87c4b650-2664-43f4-a4b2-9872468ba18b
[I 2025-04-01 01:30:52,588] Trial 0 finished with value: 0.7575983930701149 and parameters: {'eta': 0.05720986168795844, 'max_depth': 7, 'min_child_weight': 5, 'subsample': 0.377705655315485, 'colsample_bytree': 0.5226263546996813, 'gamma': 0.413952639005518, 'reg_alpha': 0.014846336063203024, 'reg_lambda': 0.004954532265527469, 'scale_pos_weight': 2.12583188232944}. Best is trial 0 with value: 0.7575983930701149.
[I 2025-04-01 01:30:52,654] Trial 1 finished with value: 0.7497395016006527 and parameters: {'eta': 0.008785498262617882, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.7495038627365594, 'colsample_bytree': 0.885083329018041, 'gamma': 0.14597638988031175, 'reg_alpha': 5.966094580064943, 'reg_lambda': 0.0029325215001713804, 'scale_pos_weight': 2.2250322728396403}. Best is trial 0 with value: 0.7575983930701149.
[I 2025-04-01 01:30:52,701] Trial 2 finished w

In [13]:
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }
    model = RandomForestClassifier(**params)
    cv_scores = cross_val_score(model, X_xgb, y_xgb, cv=5, scoring='accuracy')
    return cv_scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)


[I 2025-04-01 01:30:59,805] A new study created in memory with name: no-name-625d3913-5fc9-4cef-a465-af97b878d495
[I 2025-04-01 01:31:00,834] Trial 0 finished with value: 0.7912748728893353 and parameters: {'n_estimators': 386, 'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.7912748728893353.
[I 2025-04-01 01:31:01,361] Trial 1 finished with value: 0.7924047454648171 and parameters: {'n_estimators': 193, 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.7924047454648171.
[I 2025-04-01 01:31:03,007] Trial 2 finished with value: 0.7867804908668633 and parameters: {'n_estimators': 611, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.7924047454648171.
[I 2025-04-01 01:31:03,286] Trial 3 finished with value: 0.7879103634423451 and parameters: {'n_estimators': 101, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 1}. Best is trial 1 with value: 

In [14]:
random_forest_best_params = study.best_params
random_forest_best_score = study.best_value
xgb_best_params = xgb_study.best_params
xgb_best_score = xgb_study.best_value

print(f"Random Forest Best Parameters: {random_forest_best_params}")
print(f"Random Forest Best Score: {random_forest_best_score}")

print(f"XGB Best Parameters: {xgb_best_params}")
print(f"XGB Best Score: {xgb_best_score}")





Random Forest Best Parameters: {'n_estimators': 109, 'max_depth': 3, 'min_samples_split': 4, 'min_samples_leaf': 1}
Random Forest Best Score: 0.8013621241604418
XGB Best Parameters: {'eta': 0.24715477742304046, 'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.35675278435372115, 'colsample_bytree': 0.606919399878638, 'gamma': 0.13193866337413052, 'reg_alpha': 0.0018585361411687198, 'reg_lambda': 0.0002531352332479293, 'scale_pos_weight': 1.1316844428856405}
XGB Best Score: 0.8002510827945514


In [15]:
from optuna.importance import get_param_importances
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_param_importances(study):
    importances = get_param_importances(study)
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Bar(x=list(importances.keys()), y=list(importances.values())))
    fig.update_layout(title='Parameter Importances', xaxis_title='Parameters', yaxis_title='Importance')
    fig.show()
    

In [16]:
import os 
import pickle

os.makedirs('studies', exist_ok=True)

with open('studies/xgb_study.pkl', 'wb') as f:
    pickle.dump(xgb_study, f)

with open('studies/study.pkl', 'wb') as f:
    pickle.dump(study, f)

In [17]:
import pickle

with open('studies/xgb_study.pkl', 'rb') as f:
    xgb_study = pickle.load(f)

with open('studies/study.pkl', 'rb') as f:
    study = pickle.load(f)

In [18]:
plot_param_importances(xgb_study)
plot_param_importances(study)

In [19]:
xgb_model = xgb.XGBClassifier(**xgb_best_params,enable_categorical=True)
rf_model = RandomForestClassifier(**random_forest_best_params)

xgb_model.fit(X_xgb, y_xgb)
rf_model.fit(X_xgb, y_xgb)

RandomForestClassifier(max_depth=3, min_samples_split=4, n_estimators=109)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=3,
                                                       min_samples_split=4,
                                                       n_estimators=109)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.606919399878638,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=True,
                                              eta=0.24715477742304046,
                                              eval_me...
                                              gamma=0.13193866337413052,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=10,
                                              max_leaves=None,
                                              min_child_weight=1, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None, ...))],
                   final_estimator=LogisticRegression())

In [66]:
import optuna
import numpy as np
from sklearn.model_selection import StratifiedKFold
from models import MLP,TorchToEstimator
import torch.optim as optim

def accuracy_score(y_true,y_pred):
    return np.sum(y_true==y_pred)/len(y_true)


def objective(trial: optuna.Trial):
    params = {
        'input_size':X_xgb.shape[1],
        'hidden_size':32   ,
        'output_size':1,
        'num_layers':trial.suggest_int('num_layers', 1, 3),
        'criterion':nn.BCELoss(),
        'optimizer':optim.Adam,
        'num_epochs':trial.suggest_int('num_epochs', 100, 300)
    }
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    scores=[]
    for train_index, test_index in skf.split(X_xgb,y_xgb):
        X_train, X_test = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
        y_train, y_test = y_xgb.iloc[train_index], y_xgb.iloc[test_index]
        estimator = TorchToEstimator(**params)
        estimator.fit(X_train,y_train)
        predictions = estimator.predict(X_test)
        scores.append(accuracy_score(y_test,predictions))
    scores=np.array(scores)
    return scores.mean()


nn_study = optuna.create_study(direction='maximize')
nn_study.optimize(objective, n_trials=100)






[I 2025-04-01 12:36:48,781] A new study created in memory with name: no-name-60944164-9a07-4de1-af63-e08ae5c619e0
[I 2025-04-01 12:37:08,967] Trial 0 finished with value: 0.7890402360178269 and parameters: {'num_layers': 2, 'num_epochs': 233}. Best is trial 0 with value: 0.7890402360178269.
[I 2025-04-01 12:37:20,509] Trial 1 finished with value: 0.7070491494570333 and parameters: {'num_layers': 1, 'num_epochs': 175}. Best is trial 0 with value: 0.7890402360178269.
[I 2025-04-01 12:37:41,736] Trial 2 finished with value: 0.7968614650681063 and parameters: {'num_layers': 2, 'num_epochs': 250}. Best is trial 2 with value: 0.7968614650681063.
[I 2025-04-01 12:38:02,122] Trial 3 finished with value: 0.7957692549118072 and parameters: {'num_layers': 2, 'num_epochs': 239}. Best is trial 2 with value: 0.7968614650681063.
[I 2025-04-01 12:38:15,869] Trial 4 finished with value: 0.7687715774276568 and parameters: {'num_layers': 3, 'num_epochs': 115}. Best is trial 2 with value: 0.79686146506810

In [78]:
nn_study.best_params

{'num_layers': 3, 'num_epochs': 300}

In [79]:
nn_estimator = TorchToEstimator(**nn_study.best_params,input_size=X_xgb.shape[1],output_size=1,hidden_size=128,criterion=nn.BCELoss(),optimizer=optim.Adam)
nn_estimator.fit(X_xgb,y_xgb)


TorchToEstimator(criterion=BCELoss(), hidden_size=128, input_size=4,
                 num_epochs=300, num_layers=3,
                 optimizer=<class 'torch.optim.adam.Adam'>, output_size=1)

In [29]:
X_xgb

,Pclass,Sex,SibSp,Parch
0,3,0,1,0
1,1,1,1,0
2,3,1,0,0
3,1,1,1,0
4,3,0,0,0
...,...,...,...,...
886,2,0,0,0
887,1,1,0,0
888,3,1,1,2
889,1,0,0,0


In [30]:
y_xgb



0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [57]:
model_nn=MLP(X_xgb.shape[1],128,1,5)

X=torch.tensor(X_xgb.values,dtype=torch.float32)
y=torch.tensor(y_xgb.values,dtype=torch.float32)
dataset=torch.utils.data.TensorDataset(X,y)
dataloader=torch.utils.data.DataLoader(dataset,batch_size=128,shuffle=True)

optimizer=optim.Adam(model_nn.parameters(),lr=0.001)
criterion=nn.BCELoss()
device=torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model_nn.to(device)
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
scores=[]
model=MLP(X_xgb.shape[1],128,1,5)
model.to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001)
criterion=nn.BCELoss()
dataset=torch.utils.data.TensorDataset(X,y)
dataloader=torch.utils.data.DataLoader(dataset,batch_size=128,shuffle=True)
def train_nn(model_nn,dataloader,criterion,optimizer,num_epochs):  
    model_nn.to(device)
    model_nn.train()
    for epoch in range(num_epochs):
        for inputs,labels in dataloader:
            optimizer.zero_grad()
        outputs=model_nn(inputs.to(device))
        loss=criterion(outputs.view(-1),labels.view(-1).to(device))
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


for train_index, test_index in skf.split(torch.tensor(X_xgb.values,dtype=torch.float32),torch.tensor(y_xgb.values,dtype=torch.float32)):
    X_train, X_test = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    y_train, y_test = y_xgb.iloc[train_index], y_xgb.iloc[test_index]
    train_nn(model,dataloader,criterion,optimizer,1000)
    predictions = model(torch.tensor(X_test.values,dtype=torch.float32).to(device))
    predictions = predictions.detach().cpu().numpy()
    predictions = np.round(predictions)
    scores.append(accuracy_score(y_test.values,predictions))




Epoch 0, Loss: 0.7103521823883057
Epoch 10, Loss: 0.6329869031906128
Epoch 20, Loss: 0.6426867842674255
Epoch 30, Loss: 0.5571272969245911
Epoch 40, Loss: 0.5354892015457153
Epoch 50, Loss: 0.5092641115188599
Epoch 60, Loss: 0.5076616406440735
Epoch 70, Loss: 0.5113541483879089
Epoch 80, Loss: 0.45710891485214233
Epoch 90, Loss: 0.43839699029922485
Epoch 100, Loss: 0.4029490053653717
Epoch 110, Loss: 0.4241405129432678
Epoch 120, Loss: 0.5376607775688171
Epoch 130, Loss: 0.4078320264816284
Epoch 140, Loss: 0.47490188479423523
Epoch 150, Loss: 0.5025787353515625
Epoch 160, Loss: 0.4565812647342682
Epoch 170, Loss: 0.40228137373924255
Epoch 180, Loss: 0.4126439392566681
Epoch 190, Loss: 0.40713241696357727
Epoch 200, Loss: 0.48847582936286926
Epoch 210, Loss: 0.43561598658561707
Epoch 220, Loss: 0.5080198645591736
Epoch 230, Loss: 0.4769303798675537
Epoch 240, Loss: 0.4127008616924286
Epoch 250, Loss: 0.36652159690856934
Epoch 260, Loss: 0.3923131823539734
Epoch 270, Loss: 0.393521130084

In [58]:
scores

[np.float64(96.94413407821229),
 np.float64(95.84269662921348),
 np.float64(97.73033707865169),
 np.float64(96.31460674157303),
 np.float64(95.29213483146067)]

In [84]:
X_xgb.values


array([[3, 0, 1, 0],
       [1, 1, 1, 0],
       [3, 1, 0, 0],
       ...,
       [3, 1, 1, 2],
       [1, 0, 0, 0],
       [3, 0, 0, 0]], shape=(891, 4))

In [86]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimators = [
    ('rf', rf_model),
    ('xgb', xgb_model),
]

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)

stacking_clf.fit(X_xgb, y_xgb)

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=3,
                                                       min_samples_split=4,
                                                       n_estimators=109)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.606919399878638,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=True,
                                              eta=0.24715477742304046,
                                              eval_me...
                                              gamma=0.13193866337413052,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=10,
                                              max_leaves=None,
                                              min_child_weight=1, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None, ...))],
                   final_estimator=LogisticRegression())

In [69]:
test_data = pl.read_csv('test.csv')

test_data_xgb = test_data[list(X_xgb.columns)]

test_data_xgb = test_data_xgb.with_columns(pl.col('Sex').map_elements(lambda x: 0 if x == 'male' else 1,return_dtype=pl.Int64))

test_data_xgb.head()

Pclass,Sex,SibSp,Parch
i64,i64,i64,i64
3,0,0,0
3,1,1,0
2,0,0,0
3,0,0,0
3,1,1,1


In [70]:
predictions = stacking_clf.predict(test_data_xgb)

In [89]:
nn_predictions = nn_estimator.predict(test_data_xgb.to_pandas())

In [92]:
np.round((nn_predictions+predictions)/2)

array([0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [93]:
pl.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': predictions}).write_csv('submission.csv', separator=',')